In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

In [12]:
spark = SparkSession.builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()
spark.conf.set("spark.sql.adaptive.enabled", 'false')

In [13]:
lines = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

24/10/24 17:10:52 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [14]:
words = lines.select(
   explode(split(lines.value, " ")).alias("word")
)

In [15]:
word_counts = words.groupBy("word").count()

In [16]:
query = word_counts.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

24/10/24 17:10:57 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-00b184dd-589e-4d5c-a946-ac3649405391. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/10/24 17:10:57 ERROR MicroBatchExecution: Query [id = e25f3a3e-7ae5-4583-b7c2-95ab2e097ce6, runId = 89a9a9cc-ef7c-48c3-99f7-2f74cbd0daa3] terminated with error
java.net.ConnectException: Connection refused
	at java.base/sun.nio.ch.Net.connect0(Native Method)
	at java.base/sun.nio.ch.Net.connect(Net.java:589)
	at java.base/sun.nio.ch.Net.connect(Net.java:578)
	at java.base/sun.nio.ch.NioSocketImpl.connect(NioSocketImpl.java:583)
	at java.base/java.net.SocksSocketImpl.connect(SocksSocketImpl.java:327)
	at java.base/java.net.Socket.connect(Socket.java:751)
	at java.base/java.net.Socket.connect(Socket.java:68

In [7]:
query.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = f75b10bd-e618-4afa-b010-3337a35d66fb, runId = 7b4c7d47-618b-4201-ba26-1580970ca986] terminated with exception: Connection refused